In [1]:
# import the modules that we used 
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

In [2]:
# load the data
data=pd.read_csv('apple.csv',index_col='date')
data.tail()

,open,high,low,close,volume
date,,,,,
25/11/19,262.71,266.44,262.52,266.37,21029517
26/11/19,266.94,267.16,262.50,264.29,26334882
27/11/19,265.58,267.98,265.31,267.84,16386122
29/11/19,266.60,268.00,265.90,267.25,11654363
02/12/19,267.27,268.25,263.45,264.16,23693550


In [3]:
# get the close price
data=data[['close']]
data.head()

,close
date,
12/12/80,0.5134
15/12/80,0.4866
16/12/80,0.4509
17/12/80,0.4620
18/12/80,0.4755


In [4]:
# create a variable for n number of days for predicting the future price
ndays= 1 # n=30days
# create another variable that (target or dependent variable) shifted n days up from the row
data['prediction']=data[['close']].shift(-ndays)
print(data.head())

           close  prediction
date                        
12/12/80  0.5134      0.4866
15/12/80  0.4866      0.4509
16/12/80  0.4509      0.4620
17/12/80  0.4620      0.4755
18/12/80  0.4755      0.5045


In [5]:
# create a indipendent data set x 
# covert that data frame into numpy array
x=np.array(data.drop(['prediction'],axis=1))
print(x)

[[  0.5134]
 [  0.4866]
 [  0.4509]
 ...
 [267.84  ]
 [267.25  ]
 [264.16  ]]


In [6]:
# remove the last n rows from the data
x=x[:-ndays]
print(x)

[[  0.5134]
 [  0.4866]
 [  0.4509]
 ...
 [264.29  ]
 [267.84  ]
 [267.25  ]]


In [7]:
# create the dependent data set y
# convert the data set into numpy aray
y=np.array(data['prediction'])
# get all the y value accepted the n rows
y=y[:-ndays]
print(y)

[  0.4866   0.4509   0.462  ... 267.84   267.25   264.16  ]


In [8]:
# split the data into 80% training and 20% testing
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)


In [9]:
# create and train the support vector machine
obj=SVR(kernel='rbf',C=1e3,gamma=0.1)
obj.fit(x_train,y_train)


SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [10]:
# testing model score 
svm_score=obj.score(x_test,y_test)
print(svm_score)

0.9995992572384133


In [11]:
# create and train the linear regression model
# train the model

regr=LinearRegression()
regr.fit(x_train,y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [12]:
# testing model=score returnds the coefficient of determination R^2 of the prediction
regr_confidence=regr.score(x_test,y_test)

In [13]:
print('regr_confidence:->  ',regr_confidence)

regr_confidence:->   0.999625120072082


In [14]:
# set x_ndays equal to the last 30 rows of the originial data set from close column
x_ndays=np.array(data.drop(['prediction'],axis=1))[-ndays:]
print(x_ndays)

[[264.16]]


In [15]:
data['close'].tail() #value 264.16 is the last value of the data from the 
# close column for this value we can check the next day closing prediction price 

date
25/11/19    266.37
26/11/19    264.29
27/11/19    267.84
29/11/19    267.25
02/12/19    264.16
Name: close, dtype: float64

In [16]:
print(x_ndays)
print(data['close'].head(15))

[[264.16]]
date
12/12/80    0.5134
15/12/80    0.4866
16/12/80    0.4509
17/12/80    0.4620
18/12/80    0.4755
19/12/80    0.5045
22/12/80    0.5291
23/12/80    0.5514
24/12/80    0.5804
26/12/80    0.6339
29/12/80    0.6429
30/12/80    0.6271
31/12/80    0.6095
02/01/81    0.6161
05/01/81    0.6027
Name: close, dtype: float64


In [17]:
# print the linear model regression prediction for next 'n' days
linear_pred=regr.predict(x_ndays)
print(linear_pred)

[264.46758176]


In [18]:
# print the svr model prediction  for the next 'n' days 
svr_pred=regr.predict(x_ndays)
print(svr_pred)

[264.46758176]


In [19]:
t=[[19]]
td=pd.DataFrame(t)
linear_pred=regr.predict(td)
print(linear_pred)

[19.02176421]


In [20]:
from tkinter import *
from tkinter import messagebox as msg

In [22]:
class Linr(Frame):
    def __init__(self,master):
        super().__init__(master)
        self.l1=Label(self,text='Enter last closing price')#,font('algerian',12),bd=8,fg='light blue')
        
        self.t1=Entry(self,bd=8)#,font=('arial',14))
        
        self.b1=Button(self,text='Predict',bd=6,command=self.Show)
        self.rowconfigure(index=0,pad=10)
        self.rowconfigure(index=1,pad=10)
        
        
        
        self.l1.grid(row=1,column=0)
        self.t1.grid(row=1,column=1)
       
        self.b1.grid(columnspan=2)
        
        self.pack()
        
    def Show(self):
        w=float(self.t1.get())        
        tdata=[[x_ndays]]
        test=pd.DataFrame(tdata)
        output=regr.predict(test)
        msg.showinfo('predict', output)
        
root=Tk()
ob=Linr(root)
root.geometry('250x250')
root.mainloop()